# Import NHP surfaces from NHP-Freesurfer into Pycortex
This pipeline assumes that you have already:    
- Have [Freesurfer](https://surfer.nmr.mgh.harvard.edu/) installed.
- Processed the NHP data with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer).     
- Installed pycortex as explained in the [README](https://github.com/VisionandCognition/NHP-pycortex/blob/master/README.md) file of this repo.

If this is the case you can follow the steps below to:     
- Import the NHP-Freesurfer subject into Pycortex (surfaces)   
- Import flatmaps and transform information between templates and surfaces
- Check if it worked using dummy data    

Finally, some instructions are provided to project *any* volumetric to surfaces with Pycortex.

## Import FS subject
This works for FS subjects that have been properly processed with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). Some methods in Pycortex have been adapted to make them compatible with NHP pipelines. For example, `cortex.freesurfer.import_subjnhp` is an nhp adaptation of pycortex's `cortex.freesurfer.import_subj`. The original method is also still present, but `cortex.freesurfer.import_subjnhp` avoids trying to import data-files that do not exist for NHP-Freesurfer (like the subcortical segementation `aseg.mgz`).

In [1]:
import cortex
import os, shutil, subprocess
import numpy as np

In [2]:
subject='Eddy' # should be the FS Subject name

In [3]:
# freesurfer subject folder
fsp = os.path.join(os.environ['SUBJECTS_DIR'],subject) # if FS is installed, this env_variable should give us the subjects directory
# manual-masks path with reference volumes
manmask_path = '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks' # NB! this is system dependent
mm_func = os.path.join(manmask_path, 'sub-' + subject.lower(), 'func')
mm_anat = os.path.join(manmask_path, 'sub-' + subject.lower(), 'anat')

In [4]:
# import Freesurfer subject into pycortex
cortex.freesurfer.import_subjnhp(subject)

b'created by chris on Sun Mar 10 18:29:31 2019\n'
b'created by chris on Sun Mar 10 18:29:33 2019\n'
b'created by chris on Sun Mar 10 18:46:17 2019\n'
b'created by chris on Sun Mar 10 18:46:19 2019\n'
b'created by chris on Sun Mar 10 18:29:31 2019\n'
b'created by chris on Sun Mar 10 18:46:17 2019\n'
b'created by chris on Sun Mar 10 18:29:33 2019\n'
b'created by chris on Sun Mar 10 18:46:19 2019\n'
b'created by chris on Sun Mar 10 18:29:52 2019\n'
b'created by chris on Sun Mar 10 18:46:39 2019\n'


## Import FS subject
These flatmaps should have been generated with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). Here we only import the full hemispheric flatmap, but you can do the same thing for other flatmaps (e.g., occipital). Make sure they are in the expected naming scheme though and create copiess (like we do below) if they're not.

In [5]:
# copy FS flatmaps to expected naming scheme (may not be necessary but is if you followed NHP-Freesurfer)
for xh in ['lh','rh']:
    src_file = os.path.join(fsp,'surf', xh + '.full.patch.flat')
    trg_file = os.path.join(fsp,'surf', xh + '.full.flat.patch.3d')
    shutil.copyfile(src_file,trg_file)

In [6]:
# also import FS flatmaps into pycortex
cortex.freesurfer.import_flat(subject,"full")

flatmap vertex locations for this subject, and will result
in deletion of the overlays.svg file and all cached info
for this subject (because flatmaps will fundamentally change).
Proceed? [y]/n:  y


b'created by chris on Sun Mar 10 18:29:31 2019\n'
saving to /home/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/Eddy/surfaces/flat_lh.gii
b'created by chris on Sun Mar 10 18:46:17 2019\n'
saving to /home/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/Eddy/surfaces/flat_rh.gii
No overlays.svg file to delete


## [OPTIONAL] Create the Freesurfer based transforms
If the transforms from func/anat (reference) to the FS-anatomical on which the surfaces are based have not been created yet, do it here. If they do exist, skip these steps and move on to **Convert and import Freesurfer transforms into Pycortex**

In [7]:
# where are functional references
epi = os.path.join(mm_func, 'sub-' + subject.lower() + '_ref_func_res-1x1x1.nii.gz')
epi_mask = os.path.join(mm_func, 'sub-' + subject.lower() + '_ref_func_mask_res-1x1x1.nii.gz')
epi_brain = os.path.join(mm_func, 'sub-' + subject.lower() + '_ref_func_brain_res-1x1x1.nii.gz')
# where are anatomical references
t1 = os.path.join(mm_anat, 'sub-' + subject.lower() + '_ref_anat_res-1x1x1.nii.gz')
t1_mask = os.path.join(mm_anat, 'sub-' + subject.lower() + '_ref_anat_mask_res-1x1x1.nii.gz')
t1_brain = os.path.join(mm_anat, 'sub-' + subject.lower() + '_ref_anat_brain_res-1x1x1.nii.gz')
# where are the freesurfer anatomicals 
fs_t1 = os.path.join(fsp,'mri','brainmask.mgz')
fs_wm = os.path.join(fsp,'mri','wm.mgz')
# make a folder for reference-to-freesurfer files
mm2fs_path = os.path.join(fsp,'manual-masks_toFS') 
shutil.os.makedirs(os.path.join(mm2fs_path,'func'),exist_ok=True)
shutil.os.makedirs(os.path.join(mm2fs_path,'anat'),exist_ok=True)
# convert mgz to nifti
subprocess.run(['mri_convert', fs_t1, os.path.join(fsp,'mri','brain.nii.gz')])
subprocess.run(['mri_convert', fs_wm, os.path.join(fsp,'mri','wm.nii.gz')])

CompletedProcess(args=['mri_convert', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/mri/wm.mgz', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/mri/wm.nii.gz'], returncode=0)

Now create the registration files. First for the epi to fs_anat volumes.

In [15]:
# flirt
subprocess.run([
    'flirt', '-ref', os.path.join(fsp,'mri','brain.nii.gz'),
    '-wmseg', os.path.join(fsp,'mri','wm.nii.gz'),
    '-in', epi_brain,
    '-out', os.path.join(mm2fs_path,'func','epi2anat.nii.gz'),
    '-omat', os.path.join(mm2fs_path,'func','epi2anat.mat'),
    '-pedir','-2',
    ])

# tkreg
subprocess.run([
    'tkregister2', '--mov', epi,
    '--targ', os.path.join(fsp,'mri','brain.nii.gz'),
    '--fsl',  os.path.join(mm2fs_path,'func','epi2anat.mat'),
    '--reg',  os.path.join(mm2fs_path,'func','reg.fsl.dat'),
    '--s', subject,
    ])

CompletedProcess(args=['tkregister2', '--mov', '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks/sub-eddy/func/sub-eddy_ref_func_res-1x1x1.nii.gz', '--targ', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/mri/brain.nii.gz', '--fsl', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/manual-masks_toFS/func/epi2anat.mat', '--reg', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/manual-masks_toFS/func/reg.fsl.dat', '--s', 'Eddy'], returncode=0)

Then the anat to fs_anat (this is useful for when data is warped to anatomy). Alternatively you could also do this for a warp to NMT, but we don't here.
NB! For one of our monkeys this keeps giving weird results for anatomy. Do manual alignment if that happens.

In [30]:
# flirt
subprocess.run(['flirt',
    '-in', t1_brain,
    '-ref', os.path.join(fsp,'mri','brain.nii.gz'),
    '-out', os.path.join(mm2fs_path,'anat','anat2anat.nii.gz'),
    '-omat', os.path.join(mm2fs_path,'anat','anat2anat.mat'),
    ])

# tkreg
subprocess.run([
    'tkregister2', '--mov', t1_brain,
    '--targ', os.path.join(fsp,'mri','brain.nii.gz'),
    '--fsl',  os.path.join(mm2fs_path,'anat','anat2anat.mat'),
    '--reg',  os.path.join(mm2fs_path,'anat','reg.fsl.dat'),
    '--s', subject,
    ])

CompletedProcess(args=['tkregister2', '--mov', '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks/sub-eddy/anat/sub-eddy_ref_anat_brain_res-1x1x1.nii.gz', '--targ', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/mri/brain.nii.gz', '--fsl', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/manual-masks_toFS/anat/anat2anat.mat', '--reg', '/media/DATA1/NHP_MRI/freesurfer/subjects/Eddy/manual-masks_toFS/anat/reg.fsl.dat', '--s', 'Eddy'], returncode=0)

## Convert and import freesurfer transforms into Pycortex
These transforms have probably already been generated in [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). 

In [ ]:
# convert the existing FS transform from ref-functional to FS-anatomical into a pycortex format
xfm_name = "epi2surf" # how will we call this transform
# copy FS anatomy to expected filename (may not be necessary but is if you followed NHP-Freesurfer)
shutil.copyfile(os.path.join(fsp,'mri','T1.mgz'), os.path.join(fsp,'mri','orig.mgz'))
# set same shorter names for files
# the tkreg file from freesurfer
fs_dat_func = os.path.join(mm2fs_path,'func','reg.fsl.dat')
fs_dat_anat = os.path.join(mm2fs_path,'anat','reg.fsl.dat')

# convert
cortex.xfm.Transform.from_freesurfer(fs_dat_func,epi,subject).save(subject,xfm_name,'coord')
# and for anatomy
xfm_name = "anat2surf" # how will we call this transform
cortex.xfm.Transform.from_freesurfer(fs_dat_anat,epi,subject).save(subject,xfm_name,'coord')

In [ ]:
# You can load data with 
# test_data = <'path/to/testdata.nii.gz>'
# data = cortex.Volume.random(subject, xfm)

# Instead we'll create some random data here to test
data = cortex.Volume.random(subject, 'epi2surf')

In [ ]:
# open web viewer
cortex.webshow(data)